In [13]:
import os
os.chdir('/home/arjun/text_summerization_end-end/text_summarizer')
%pwd

'/home/arjun/text_summerization_end-end/text_summarizer'

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)  
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path
    

In [15]:
from src.text_summarizer.constant import *
from src.text_summarizer.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [17]:
import os
from src.text_summarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [18]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))


In [19]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-05-22 11:43:18,595: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-05-22 11:43:18,596: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-22 11:43:18,597: INFO: common: created directory at: artifacts]
[2024-05-22 11:43:18,622: INFO: common: created directory at: artifacts/data_transformation]


/home/arjun/text_summerization_end-end/text_summarizer/text-env/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/home/arjun/text_summerization_end-end/text_summarizer/text-env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 137089.57 examples/s]
